In [ ]:
# ! pip install git+https://github.com/fraenkel-lab/OmicsIntegrator2.git
# ! pip install --upgrade --force-reinstall --no-deps python-louvain
# ! pip install --upgrade --force-reinstall --no-deps networkx

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt

import OmicsIntegrator as oi


In [17]:
interactome_file = "interactome.txt"
prize_file = "inputs\Covid_prizes.txt"
plasma_proteins = "inputs\plasma_proteins_atlas.csv"

interactome = pd.read_csv(interactome_file, delimiter = '\t')
plasma = pd.read_csv(plasma_proteins)

out1 = pd.merge(interactome, plasma, left_on= 'protein1', right_on= 'Gene', how='inner')
out2 = pd.merge(interactome, plasma, left_on= 'protein2', right_on= 'Gene', how='inner')
plasma_interactome = pd.concat([out1,out2]).drop_duplicates()[['protein1', 'protein2', 'cost']].to_csv('plasma_interactome.txt', sep='\t', index= False)





graph = oi.Graph('plasma_interactome.txt')

#{'g':5, "w":5 , "b":7}

In [18]:
graph.prepare_prizes(prize_file)

#Ws = list(np.arange(0,5,1))
#Bs = list(np.arange(0,5,1))
#Gs = list(np.arange(0.5,10,1))

# results = graph.grid_search(prize_file, Ws, Bs, Gs)
# membership_df = oi.summarize_grid_search(results, "membership")

11:29:48 - OI2: INFO - Duplicated gene symbols in the prize file (we'll keep the max prize):
11:29:48 - OI2: INFO - []
11:29:48 - OI2: INFO - Members of the prize file not present in the interactome:
11:29:48 - OI2: INFO - ['C4A', 'FCGBP', 'IGHG2', 'IGHG4', 'IGHV3-43', 'IGHV3-72', 'IGHV3-74', 'IGKV1D-13', 'IGKV1D-37', 'IGKV2-40', 'IGKV3-20', 'IGKV3D-15', 'IGKV4-1', 'IGLV1-40', 'IGLV3-21', 'IGLV6-57', 'KRTDAP', 'TMEM198']


In [19]:
vertex_indices, edge_indices = graph.pcsf()

In [20]:
len(vertex_indices)

128

In [21]:
forest, augmented_forest = graph.output_forest_as_networkx(vertex_indices, edge_indices)
print(augmented_forest)

Graph with 128 nodes and 303 edges


In [ ]:
nx.draw_spring(forest)

In [23]:
graph.pcsf_objective_value(augmented_forest)

118.64054607200002

In [24]:
oi.get_networkx_graph_as_dataframe_of_edges(augmented_forest).head()

,protein1,protein2,in_solution,cost
0,CFI,CFH,True,0.205775
1,CFI,C3,NaN,0.675872
2,FCGR3A,APCS,NaN,0.438873
3,FCGR3A,CD4,True,0.234063
4,FCGR3A,GP6,True,0.205775


In [ ]:
oi.output_networkx_graph_as_graphml_for_cytoscape(augmented_forest, output_dir='')

In [26]:
oi.get_networkx_graph_as_dataframe_of_edges(augmented_forest).to_csv('edges.txt', sep='\t', header=True, index=False)
oi.get_networkx_graph_as_dataframe_of_nodes(augmented_forest).to_csv('nodes.txt', sep='\t', header=True, index=True)


In [ ]:
Data = pd.read_csv('interactome.txt', delimiter='\t')
file = open('shortest_path.csv', 'w')

src = pd.read_csv('nodes.txt', delimiter='\t')
src = src.iloc[:,0].values

trgt = pd.read_csv('gordon_proteins.txt', delimiter='\t')
trgt = trgt.iloc[:,2].values

G = nx.from_pandas_edgelist(Data, edge_attr='cost', source='protein1', target = 'protein2')
G.remove_node('UBC') 

for s in src:
  for t in trgt:
    
    try:
      sp=nx.shortest_path(G, source=s, target=t)
      if len(sp)<= 2:
        file.writelines('\t'.join(sp)+'\n')
        print(sp)
    except:
      continue
file.close()

  
